<a href="https://colab.research.google.com/github/alisohaila/AWS-DynamoDB-Capacity-Units-Calculator/blob/main/Copy_of_N_Gram_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-Gram Model
An n-gram model generates an original text inspired by existing texts.
The main idea behind the n-gram model is to predict the next word in a text based on the previous n words. To build your model, you will need to understand the following terminology:
  

*   **Corpus**: A collection of texts. Generally, researchers curate corpora using texts with a common theme. For instance, there are corpuses of Reddit comments and Shakespearean plays.
*   **Lexicon**: A vocabulary of words and punctuation in a corpus. In the corpus, capitalization in words does not matter. The punctuation is described in the parse_story function. 
*   **Token**: A single element of the lexicon.
*   **N-gram**: A series of N tokens that appear consecutively in the corpus, where N is the number of these consecutive tokens in the N-gram (examples follow).

Consider the short corpus below:

Once upon a time there was a child. The child is sad today. However, the child will go out to play, and the child can not be sad anymore.


*   Examples of 2-grams include: (once, upon), (upon, a), (a, time), … ,(sad, anymore), (anymore, .) 
*   Examples of 3-grams include: (once, upon, a), (., the, child), (to, play, ,), (sad, anymore, .)


## Task Ahead

Your text generation will consider the probability of a specific token following an N-gram and pick
a word/punctuation based on those probabilities. To generate the probabilities, you will parse
the corpus and count the different occurrences of a single word after an N-gram. Returning to
the short corpus using 2-grams, the 2-gram (once, upon) is followed by “a”. The 2-gram (the,
child) is followed once by “is”, once by “will”, and once by “can”. Thus, the next word after “once
upon” will be “a” with 100% probability. The next word after “the child” will be one of “is”, “will”,
and “can”, each with one-thirds probability.

In [ ]:
from utilities import *

### Function 1

`parse_story(filename : str) -> list`

Returns an ordered list of words with bad characters processed (removed) from the text in the file given by `file_name`.

In [ ]:
def parse_story(filename):
  storyfile = open(filename, 'r')
  content = storyfile.read()
  content = content.lower()
  output = []
  word = ''

  for i in range(len(content)):
    if content[i] in [" ", '\n', '\t'] and word != '':
      if word.capitalize() in ALWAYS_CAPITALIZE:
        word = word.capitalize()
      output.append(word)
      word = ''
  
    elif content[i] in VALID_PUNCTUATION:
      if word != '':
        if word.capitalize() in ALWAYS_CAPITALIZE:
          word = word.capitalize()
        output.append(word)
      output.append(content[i])
      word = ''

    elif content[i] not in (BAD_CHARS) and content[i] not in [" ", '\n', '\t']:
      word = word + content[i]

  storyfile.close()
  return output

In [ ]:
words = parse_story('308.txt')

### Function 2

`get_prob_from_count(counts : list) -> list`

Return a list of probabilities derived from counts. Counts is a list of counts of occurrences of a token after the previous n-gram. You should not round the probabilities.

In [ ]:
def get_prob_from_count(counts):
  summation = sum(counts)
  output = []

  for element in counts:
    output.append(element/summation)
  
  return output

In [ ]:
get_prob_from_count([10,20,30,40,50])

### Function 3

`build_ngram_counts(words : list, n : int) -> dict`

Return a dictionary of N-grams (where N=`n`) and the counts of the words in the list `words` that follow the Ngram. The key of the dictionary will be the N-gram in a tuple. The corresponding value will be a list containing two lists. The first list contains the words and the second list contains the corresponding counts.

Use parse_story function to test

In [ ]:
def build_ngram_counts(words, n):
  ngram_dict = {}
  for i in range(len(words) - n):

    if tuple(words[i : i + n]) not in ngram_dict:
      ngram_dict[tuple(words[i : i + n])] = [ [words[i+n]], [1] ]

    else:
      collection = ngram_dict[tuple(words[i : i + n])]
      
      if words[i+n] in collection[0]:
        index = collection[0].index(words[i+n])
        collection[1][index] += 1
        ngram_dict[tuple(words[i : i + n])] = collection
      
      else:
        collection[0].append(words[i+n])
        collection[1].append(1)
        ngram_dict[tuple(words[i : i + n])] = collection
    
  return ngram_dict

In [ ]:
counts = build_ngram_counts(words, 2)

### Function 4

`prune_ngram_counts(counts : dict, prune_len : int) -> dict`

Return a dictionary of N-grams and counts of words with lower frequency (i.e. occurring less
often) words removed. You will prune the words based on their counts, keeping the
prune_len highest frequency words. In case of a tie (for example, if `prune_len` was 5 and the 5th and 6th most frequent words had the same frequency), then keep all words that are in the tie (e.g. keep both the 5th and 6th words).

Use previous functions here.

In [ ]:
def prune_ngram_counts(counts, prune_len):
	'''
	(dict, int) -> (dict)
	Return a dictionary of N-grams and counts of words with lower frequency (i.e. occurring less often) words removed. 
	>>> ngram_counts= {(‘i’, ‘love’): [[‘js’, ‘py3’, ‘c’, ‘no’], [20, 20, 10, 2]],(‘u’, ‘r’): [[‘cool’, ‘nice’, ‘lit’, 'kind’], [8, 7, 5, 5]],('toronto’, ‘is’): [[‘six’, ‘drake’], [2, 3]]}
	>>> prune_ngram_counts(ngram_counts, 3)
	{(‘i’, ‘love’): [[‘js’, ‘py3’, ‘c’], [20, 20, 10]],(‘u’, ‘r’): [[‘cool’, ‘nice’, ‘lit’, 'kind’], [8, 7, 5, 5]],('toronto’, ‘is’): [[‘six’, ‘drake’],[2, 3]]}
	'''
	for ngram, lists in counts.items():
		if len(lists[0]) > prune_len:
			min_count = lists[1][1]
			for j in lists[1]:
				if j < min_count:
					min_count = j
			if lists[1].count(min_count) <= (len(lists[0]) - prune_len):
				for k in range(lists[1].count(min_count)):
					ind = lists[1].index(min_count)
					lists[1].remove(min_count)
					lists[0].pop(ind)
				if len(lists[0]) > prune_len:
					prune_ngram_counts({ngram : lists}, prune_len)
			else:
				counts.update({ngram : lists})
	return counts

### Function 5

`probify_ngram_counts(counts : dict) -> dict`

Take a dictionary of N-grams and counts and convert the counts to probabilities. The probability of each word is defined as the observed count divided by the total count of all words.

In [ ]:
def probify_ngram_counts(counts):
  for ngram, lists in counts.items():
    lists[1] = get_prob_from_count(lists[1])
    counts[ngram] = lists
  return counts

In [ ]:
probify_ngram_counts(counts)

### Function 6

`build_ngram_model(words, n)`

Create and return a dictionary of the format given above in probify_ngram_counts. This dictionary is your final model that will be used to auto-generate text. For your final model, keep the 15 most likely words that follow an N-gram. 

_Challenge_: Moreover, for each N-gram, the corresponding next words should appear in descending order of probability.

In [ ]:
def build_ngram_model(words, n):
	'''
	(list, int) -> (dict)
	
	'''
	temp1 = {}
	temp1.update(build_ngram_counts(words, n))
	temp1.update(prune_ngram_counts(temp1, 15))
 
  # Challenge Part=============================
	for ngram, lists in temp1.items():
		lists[1], lists[0] = zip(*sorted(zip(lists[1], lists[0]), reverse = True))
		lists[1] = list(lists[1])
		lists[0] = list(lists[0])
  # ===========================================

	temp1.update(probify_ngram_counts(temp1))
	return temp1

### Function 7

`gen_bot_list(ngram_model, seed, num_tokens=0)`

Returns a randomly generated list of tokens (strings) that starts with the N tokens in seed, selecting all subsequent tokens using gen_next_token. The list ends when any of the following happens:


*   List contains num_tokens tokens, including repetitions. In case seed is longer than num_tokens, the returned list should contain the first num_tokens tokens of the seed.
*   Any of the assumptions of gen_next_token is violated. I.e. if either an N-gram is not
in the model, or if an N-gram has no tokens that follow it.

In [ ]:
def gen_bot_list(ngram_model, seed, num_tokens = 0):
	'''
	(dict, tuple, int) -> (list)
	
	'''
	if num_tokens == 0:
		return []
	else:
		if len(seed) >= num_tokens:
			return list(seed[ : num_tokens])
		else:
			if seed not in ngram_model:
				return list(seed)
			else:
				sentence = list(seed)
				for i in range(num_tokens - len(seed)):
					if tuple(sentence[i : i + len(seed)]) in ngram_model and len(sentence) < num_tokens:
						word = gen_next_token(tuple(sentence[i : i + len(seed)]), ngram_model)
						sentence.append(word)
					else:
						return sentence
				return sentence

### Function 8

`gen_bot_text(token_list, bad_author)`

If bad_author is True, returns the string containing all tokens in token_list, separated by a space. Otherwise, returns this string of text, respecting the following grammar rules:

*   There are no spaces before tokens found in `VALID_PUNCTUATION`
*   A sentence cannot start with a lower-case letter. The array in `utilities.py` called `END_OF_SENTENCE_PUNCTUATION` tells you how to determine that the previous sentence has finished. (Recall that strings are case-sensitive).
*   Words in `ALWAYS_CAPITALIZE` should start with a capital letter

In [ ]:
def gen_bot_text(token_list, bad_author):
	'''
	(list, bool) -> (str)
	
	'''
	string = ''
	if bad_author:
		for word in token_list:
			string += word
			string += ' '
		return string
	else:
		caps = False
		capitals = []
		for i in range(len(ALWAYS_CAPITALIZE)):
			capitals.append(ALWAYS_CAPITALIZE[i].lower())
		for word in token_list:
			if len(string) == 0:
				string += word.capitalize()
			else:
				if word in VALID_PUNCTUATION:
					caps = False
					if word in END_OF_SENTENCE_PUNCTUATION:
						caps = True
					string += word
				elif word in capitals:
					Word = word.capitalize()
					string += ' ' + Word
					caps =False
				else:
					if caps:
						Word = word.capitalize()
						string += ' ' + Word
						caps = False
					else:
						string += ' '+ word  	
		return string

### Function 9

`write_story(file_name, text, title, student_name, author, year)`

Writes the text to the file with name file_name. The file should begin with a title “page”, starting with 10 newline chars and ending with 17 newline chars, and having the following content:

**title**: year, UNLEASHED

**student_name**, inspired by author

Copyright year published (year), publisher: GEC press

For exact formatting and spacing between characters refer to test_write_story.txt or test_gen_bot_text.txt.

You can comfortably read about 90 characters on a line (excluding new line character from the count). Place the largest number of words you can fit on the 90-character line. Do not break words or any other sequences of characters that were not initially separated by white space. Do not begin or end a line in a space character.

When GEC press prints the text, it can fit exactly 30 lines on a page. Place a page number as the last line of each page (starting with 1 and the title page is not numbered). The line before the page number should be blank. The last page should also display the page number on the 30th line (no new line character at the end of the file)

Every 12 pages is a new chapter. Write chapter headings followed by two newline characters:

CHAPTER #\n\n

starting with CHAPTER 1\n\n

In [ ]:
def write_story(file_name, text, title, student_name, author, year):
	'''
	(string, string, string, string, string, int) -> (file)
	
	'''
	test = open(file_name, 'w')
	L = text.split()
	content = 'CHAPTER 1\n\n'
	count = 0
	count_line = 2
	count_page = 0
	count_chap = 1
	for word in L:
		if content == 'CHAPTER 1\n\n':
			content += word
			count += len(word)
		else:	
			count += len(word) + 1
			if count > 90:
				count_line += 1
				if count_line == 28:
					count_line = 0
					count_page += 1
					content += '\n\n' + str(count_page)
				if count_page % 12 == 0 and count_page != 0 and count_line == 0 and word != L[-1]:
					count_chap += 1
					content += '\nCHAPTER ' + str(count_chap) + '\n'
					count_line = 2
				count = len(word)
				content += '\n' + word	
			else: 
				content += " " + word 
	
	if count_line != 0:
		for i in range(29 - count_line):
			content += '\n'
		content += str(count_page+1) + '\n'
	test.write('\n\n\n\n\n\n\n\n\n\n'+title+': '+str(year)+', UNLEASHED\n'+student_name+', inspired by '+author+'\nCopyright year published ('+str(year)+'), publisher: EngSci press\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'+content)
	test.close()